In [ ]:
# !wget https://www.manythings.org/anki/rus-eng.zip && unzip rus-eng.zip

### Домашнее задание Transformers Training (50 баллов)

В этом домашнем задании требуется обучить несколько Transformer-based моделей в задаче машинного перевода. Для обучения можно воспользоваться текущим проектом, так и реализовать свой пайплайн обучения. Если будете использовать проект, теги **TODO** проекта отмечают, какие компоненты надо реализовать.
В ноутбуке нужно только отобразить результаты обучения и выводы. Архитектура модели(количетсво слоев, размерность и тд) остается на ваш выбор.

Ваш код обучения нужно выложить на ваш github, в строке ниже дать ссылку на него. В первую очередь будут оцениваться результаты в ноутбуке, код нужен для проверки адекватности результатов. 

Обучать модели до конца не нужно, только для демонстрации, что модель обучается и рабочая - снижение val_loss, рост bleu_score.

#### Сcылка на ваш github с проектом(вставить свой) - https://github.com/runnerup96/pytorch-machine-translation

Ноутбук с результатами выкладывать на ваш **google диск** курса. 

### Данные

`
wget https://www.manythings.org/anki/rus-eng.zip && unzip rus-eng.zip
`

Модели нужно обучить на задаче перевода с английского на русский. 

### Обучение Seq2seq Transformer модель(25 баллов)

Реализуйте Seq2seq Transformer. В качестве блока трансформера можно использовать https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html. В качестве токенизатора воспользуйтесь HuggingFace токенизатор для source/target языков - https://huggingface.co/docs/transformers/fast_tokenizers
В качестве максимальной длинны возьмите предложения длинной **до 15 слов**, без каких либо префиксов. 

Не забудьте остальные элементы модели:
* Мы можем использовать 1 трансформер как энкодер - декодером будет выступать линейный слой. 
* Обучите свой BPE токенизатор - https://huggingface.co/docs/transformers/fast_tokenizers
* Матрицу эмбеддингов токенов
* Матрицу позицонных эмбеддингов
* Линейный слой проекции в target словарь
* Функцию маскирования будущих состояний attention, так как модель авто-регрессионна
* Learning rate schedualer


В качестве результатов, приложите слудующие данные:
1) Параметры обучения - learning rate, batch_size, epoch_num, размерность скрытого слоя, количетсво слоев
2) Графики обучения - train loss, val loss, bleu score
3) Примеры переводов вашей модели(10 штук) - source text, true target text, predicted target text

### Results

Представлено две реализации, с разным подходом к обучению
 - `seq2seq_transformer` через forward на src + tgt
 - `seq2seq_transformer2` через forward на src, в predict формате

last checkpoint path:```./Transformer-model-project/eng2ru-transformer-translator-0.054bleu.ckpt```

### Load Model

#### Imports

In [2]:
import sys, os
import importlib
import torch
sys.path.append(os.path.join(os.getcwd(), "./Transformer-model-project/src"))

from data.datamodule import DataManager

device = torch.device("mps" if torch.cuda.is_available() else "cpu")

In [3]:
eng_prefixes = (
    "i am ",
    "i m ",
    "he is",
    "he s ",
    "she is",
    "she s ",
    "you are",
    "you re ",
    "we are",
    "we re ",
    "they are",
    "they re ",
)

def filter_func(x):
    MAX_LENGTH = 15
    len_filter = lambda x: len(x[0].split(" ")) <= MAX_LENGTH and len(x[1].split(" ")) <= MAX_LENGTH
    eng_prefix_filter = lambda x: x[0].startswith(eng_prefixes)
    rus_prefix_filter = lambda x: x[0].startswith(rus_prefixes)
    return len_filter(x) and prefix_filter(x)

config = {
    "batch_size": 64,          # <--- size of batch
    "num_workers": 47,          # <--- num cpu to use in dataloader
    "prefix_filter": eng_prefixes,      # <--- callable obj to filter data
    "max_length": 15,
    "filename": "./rus.txt",    # <--- path to file with sentneces
    "lang1": "en",              # <--- name of the first lang    
    "lang2": "ru",              # <--- name of the second lang
    "reverse": False,           # <--- direct or reverse order in pairs
    "train_size": 0.8,          # <--- ratio of data pairs to use in train
    "run_name": "tutorial",     # <--- run name to logger and checkpoints
    "quantile": 0.95,           # <--- (1 - quantile) longest sentences will be removed
}

In [4]:
dm = DataManager(config, device)
dm.prepare_data()

Reading from file: 100%|██████████| 496059/496059 [00:03<00:00, 164368.64it/s]





Space tokenizer fitted - 18831 tokens



Space tokenizer fitted - 30000 tokens


(<bound method DataManager.train_dataloader of <data.datamodule.DataManager object at 0x14c68b620>>,
 <bound method DataManager.val_dataloader of <data.datamodule.DataManager object at 0x14c68b620>>)

#### Model checkpoint

In [5]:
from models import seq2seq_transformer2
importlib.reload(seq2seq_transformer2)

<module 'models.seq2seq_transformer2' from '/Users/a.i.krotov/Desktop/universal_osa/contest/hw-2/./Transformer-model-project/src/models/seq2seq_transformer2.py'>

In [61]:
checkpoint = "Transformer-model-project/eng2ru-transformer-translator-0.054bleu.ckpt"
model = seq2seq_transformer2.Seq2SeqTransformer.load_from_checkpoint(
       checkpoint,
               lr = 1e-3,
            nhead = 4,
          src_dim = dm.input_lang_n_words,  # SV
          tgt_dim = dm.output_lang_n_words, # TV
          emb_dim = 256,
          hdn_dim = 256, # dim_feedforward
      enc_nlayers = 3,
      dec_nlayers = 3,
    tgt_tokenizer = dm.target_tokenizer,
    src_tokenizer = dm.source_tokenizer,
          dropout = 0.2,
          max_len = 15,
      tgt_pad_idx = dm.target_tokenizer.word2index['PAD'],
      tgt_sos_idx = dm.target_tokenizer.word2index['SOS'],
      tgt_eos_idx = dm.target_tokenizer.word2index['EOS'],
      src_pad_idx = dm.source_tokenizer.word2index['PAD'],
      src_sos_idx = dm.source_tokenizer.word2index['SOS'],
      src_eos_idx = dm.source_tokenizer.word2index['EOS']
)

### Параметры обучения модели
```python
      # learning rate = 1e-3
         # batch size = 128
          # epoch num = 30
         # hidden dim = 256
# encoding num layers = 3
# decoding num layers = 3
```

### Логи метрик

Вы взаимодействуете с Моделью №71

In [64]:
# !tensorboard --logdir=./Transformer-model-project/lightning_logs --port=6006

### Примеры перевода

In [62]:
import random
idx_list = [random.randint(0, len(dm.source_sentences)-1) for _ in range(10)]
idx_list

[56848, 258341, 297378, 43599, 173832, 122408, 610, 240124, 86545, 199174]

In [63]:
def translate(phrase):
    model.eval()
    prediction = model.predict(phrase)
    model.train()
    return prediction

for idx in idx_list:
    print('eng: ', dm.source_sentences[idx])
    print('rus target: ', dm.target_sentences[idx])
    print('rus predicted: ', translate(dm.source_sentences[idx]))
    print()
    

eng:  does that satisfy you
rus target:  тебя это устраивает
rus predicted:  это удовлетво удовлетво удовлетво удовлетво удовлетво удовлетво

eng:  you only have to follow the instructions
rus target:  вам надо просто следовать инструкциям
rus predicted:  тебе должен следовать инструкциям инструкциям инструкциям инструкциям

eng:  i suppose you know why tom didn t do what we asked him to do
rus target:  я полагаю вы знаете почему том не сделал то что мы просили его сделать
rus predicted:  полагаю почему почему почему почему почему почему почему почему почему почему почему почему почему

eng:  tom says he ll come
rus target:  том говорит что придёт
rus predicted:  том говорит что придёт

eng:  tom is naked from the waist up
rus target:  том голый по пояс
rus predicted:  том голый голый голый пояс

eng:  this is a question for tom
rus target:  это вопрос к тому
rus predicted:  это вопрос вопрос вопрос вопрос вопрос

eng:  sit still
rus target:  сиди спокойно
rus predicted:  садись ещё

e

### Fine-tune pretrained T5 (25 баллов)

Реализуйте Seq2seq Pretrained T5. Воспользуйтесь https://huggingface.co/docs/transformers/model_doc/t5 предобученной моделью. В качестве максимальной длинны возьмите предложения длинной **до 15 слов**, без каких либо префиксов. Архитектура модели(количетсво слоев, размерность и тд) остается на ваш выбор.

Не забудьте важные аспекты обучения модели:
* Взять готовый t5 токенизатор
* Resize matrix embedding - скорей всего ваша матрица эмбеддингов не будет включать эмбеддинги из вашего сета. Пример обновления матрицы эмбеддингов тут тут https://github.com/runnerup96/Transformers-Tuning/blob/main/t5_encoder_decoder.py
* Learning rate schedualer/Adafactor with constant learning rate


В качестве результатов, приложите слудующие данные:
1) Параметры обучения - learning rate, batch_size, epoch_num, pretrained model name
2) Графики обучения - train loss, val loss, bleu score
3) Примеры переводов вашей модели(10 штук) - source text, true target text, predicted target text

### Results

last checkpoint path:```./T5-model-project/eng2ru-t5-translator-0.2bleu.ckpt```

### Load Model

#### Imports

In [ ]:
import sys, os
import importlib
import torch
sys.path.append(os.path.join(os.getcwd(), "./T5-model-project/src_t5"))

from data.datamodule import DataManager

device = torch.device("cuda:6" if torch.cuda.is_available() else "cpu")

#### Tokenizer

In [ ]:
eng_prefixes = (
    "i am ",
    "i m ",
    "he is",
    "he s ",
    "she is",
    "she s ",
    "you are",
    "you re ",
    "we are",
    "we re ",
    "they are",
    "they re ",
)

def filter_func(x):
    MAX_LENGTH = 15
    len_filter = lambda x: len(x[0].split(" ")) <= MAX_LENGTH and len(x[1].split(" ")) <= MAX_LENGTH
    eng_prefix_filter = lambda x: x[0].startswith(eng_prefixes)
    rus_prefix_filter = lambda x: x[0].startswith(rus_prefixes)
    return len_filter(x) and prefix_filter(x)

config = {
    "batch_size": 64,          # <--- size of batch
    "num_workers": 47,          # <--- num cpu to use in dataloader
    "prefix_filter": eng_prefixes,      # <--- callable obj to filter data
    "max_length": 15,
    "filename": "./rus.txt",    # <--- path to file with sentneces
    "lang1": "en",              # <--- name of the first lang    
    "lang2": "ru",              # <--- name of the second lang
    "reverse": False,           # <--- direct or reverse order in pairs
    "train_size": 0.8,          # <--- ratio of data pairs to use in train
    "run_name": "tutorial",     # <--- run name to logger and checkpoints
    "quantile": 0.95,           # <--- (1 - quantile) longest sentences will be removed
}

In [ ]:
dm = DataManager(config, device)
dm.prepare_data()

Reading from file: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 496059/496059 [00:05<00:00, 83608.40it/s]
/home/krotovan/hw-sber/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have bee

(<bound method DataManager.train_dataloader of <data.datamodule.DataManager object at 0x7f5d4fda5000>>,
 <bound method DataManager.val_dataloader of <data.datamodule.DataManager object at 0x7f5d4fda5000>>)

#### Model checkpoint

In [ ]:
from models import seq2seq_t5
importlib.reload(seq2seq_t5)

<module 'models.seq2seq_t5' from '/home/krotovan/hw-sber/pytorch-project/./T5-model-project/src_t5/models/seq2seq_t5.py'>

In [ ]:
checkpoint = "./T5-model-project/eng2ru-t5-translator-0.2bleu.ckpt"
model = seq2seq_t5.Seq2SeqT5.load_from_checkpoint(checkpoint, tokenizer=dm.tokenizer)

### Параметры обучения модели
```python
        # learning rate = 1e-3
           # batch size = 64
            # epoch num = 8
# pretrained model name = "google-t5/t5-small"
```

### Логи метрик

In [ ]:
# !tensorboard --logdir=./T5-model-project/lightning_logs --port=6006

### Примеры перевода

In [ ]:
import random
idx_list = [random.randint(0, len(dm.source_sentences)-1) for _ in range(10)]
idx_list

[251088, 210897, 224177, 117624, 124101, 240902, 99409, 194512, 121185, 271286]

In [ ]:
def translate(phrase):
    model.eval()
    in_tokens = model.tokenizer(phrase)
    in_tokens_input_ids      = torch.Tensor([in_tokens.input_ids     ]).to(model.device).long()
    in_tokens_attention_mask = torch.Tensor([in_tokens.attention_mask]).to(model.device).long()
    prediction = model.predict(in_tokens_input_ids, in_tokens_attention_mask)
    model.train()
    return model.tokenizer.decode(prediction[0], skip_special_tokens=True)
    
for idx in idx_list:
    print('eng: ', dm.source_sentences[idx])
    print('rus target: ', dm.target_sentences[idx])
    print('rus predicted: ', translate(dm.target_sentences[idx]))
    print()
    

eng:  tom and mary said they were on our side
rus target:  том и мэри сказали что они на нашей стороне
rus predicted:  том  с скаали

eng:  i hope i can pass the driving test
rus target:  надеюсь я смогу сдать экзамен по вождению
rus predicted:  надес  смоу сда

eng:  tom put the thermometer on the wall
rus target:  том повесил термометр на стену
rus predicted:  том том овесил т

eng:  i hate standing on the bus
rus target:  ненавижу стоять в автобусе
rus predicted:  стот в автоусе

eng:  tom laughed good naturedly
rus target:  том добродушно рассмеялся
rus predicted:  том том дораи

eng:  call me immediately after you meet him
rus target:  позвони мне сразу как только с ним встретишься
rus predicted:  овони мне не сра

eng:  tom says that he s tired
rus target:  том говорит что устал
rus predicted:  том оворит то

eng:  the money on the desk isn t mine
rus target:  деньги на столе не мои
rus predicted:  дении на столеле не

eng:  she reached out for my arm
rus target:  она потянулась 

#### [мем к переводу](https://upload.wikimedia.org/wikipedia/commons/thumb/7/7e/%D0%AD%D1%82%D0%BE_%D0%BD%D0%B5%D0%BC%D0%BD%D0%BE%D0%B3%D0%BE%2C_%D0%BD%D0%BE_%D1%8D%D1%82%D0%BE_%D1%87%D0%B5%D1%81%D1%82%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0.jpg/800px-%D0%AD%D1%82%D0%BE_%D0%BD%D0%B5%D0%BC%D0%BD%D0%BE%D0%B3%D0%BE%2C_%D0%BD%D0%BE_%D1%8D%D1%82%D0%BE_%D1%87%D0%B5%D1%81%D1%82%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0.jpg?20220728114737)